## Generate a Synthetic dataset

- model used : Qwen3-0.6B
- model uses a few shot prompt technique to understand the query of the customer and then converts the query into a `support ticket vector` for the customer support process.
- I have used ChatGPT to create 40 customer queries and later use these customer queries are passed to the LLM Qwen3-0.6B to generate a default formatted Support Ticket.
- Used Few-Shot Prompts to generate a Support Ticket via the LLM
- Parse the string created by the LLM and convert it into an actual JSON object
- store the queries and the support ticket into a csv file for later use as a vector db

In [1]:
"""Generating a synthetic dataset"""

import numpy as np
import pandas as pd
import json


In [2]:
# Sample Cutomer Query
customer_query = """Hi there,
We're experiencing issues with file uploads in your web app. Whenever we try to upload a PDF larger than 5MB, it fails silently—no error message, just nothing happens. This occurs on both Edge (v120) and Chrome (v121) on Windows 11. Smaller files seem to upload fine. Can you look into this?

Best,
Ayesha (IT Admin, Acme Corp)"""

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import re
def generate_support_ticket(query):

    model_name = "Qwen/Qwen3-1.7B"

    # load the tokenizer and the model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype="auto",
        device_map="auto"
    )

    customer_query = query
    prompt = f"""
    You are a structured support assistant. Your role is to extract key information from customer queries and generate a suggested resolution. Return everything as a properly formatted JSON object.

    Instructions:
    - Output only the JSON object — no additional text, labels, or explanations.
    - Ensure the JSON is syntactically valid: use double quotes, proper commas, and braces.
    - The "Resolution" field must contain a reasonable fix or suggestion based on the described problem.
    - The "Resolution" field is mandatory — infer a likely troubleshooting step or recommendation, even if it’s not explicitly stated in the query.
    - Try to fill in as many relevant values as possible based on the query. If you are unsure about a value for any key, set it to null.
    - If the customer value is null, make it 'Individual'

    Fields to extract:
    {{
        "Query": "Full query with all greetings and names removed",
        "Title": "One-line summary of the issue",
        "Browser": "Browser name and version if mentioned",
        "OS": "Operating system if mentioned",
        "Customer Type": "Individual, Enterprise, Developer, etc.",
        "Issue": "Short description of the core problem",
        "Resolution": "Recommended troubleshooting step or fix"
    }}

    Examples:

    Example 1:
    Query:
    "
    Hi,
    I'm having trouble logging into our account using Single Sign-On (SSO) on Safari (version 16.3) on macOS Ventura. When I try to log in, I'm stuck in a redirect loop and never reach the dashboard. This issue doesn't occur on other browsers. Could you please help me resolve this?

    Thanks,
    John
    "

    Response:
    {{
        "Query": "I'm having trouble logging into our account using Single Sign-On (SSO) on Safari (version 16.3) on macOS Ventura. When I try to log in, I'm stuck in a redirect loop and never reach the dashboard. This issue doesn't occur on other browsers.",
        "Title": "Login failure on Safari for SSO users",
        "Browser": "Safari 16.3",
        "OS": "macOS Ventura",
        "Customer Type": "Enterprise",
        "Issue": "Redirect loop during SSO login",
        "Resolution": "Clear cookies and enable cross-site tracking in Safari settings"
    }}

    Example 2:
    Query:
    "
    Hello team,
    Our developer portal isn't loading correctly in Chrome. We see a blank white screen after logging in. No errors appear, but the console shows some CORS-related messages. We're accessing from Windows 10.

    Thanks,
    Kira (DevOps Engineer)
    "

    Response:
    {{
        "Query": "Our developer portal isn't loading correctly in Chrome. We see a blank white screen after logging in. No errors appear, but the console shows some CORS-related messages. We're accessing from Windows 10.",
        "Title": "Developer portal fails to load in Chrome",
        "Browser": "Chrome (version not specified)",
        "OS": "Windows 10",
        "Customer Type": "Developer",
        "Issue": "Blank screen and CORS warnings post-login",
        "Resolution": "Check CORS headers on the server and suggest using the latest browser version"
    }}

    Now process the following query and return only a valid JSON object:

    Query:
    {customer_query}
    """


    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # conduct text completion
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=32768
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0

    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
    
    return content



### Import customer queries csv file
- Load the customer queries from the `customer queries.csv` file into a dataset variable using pandas
- Convert the pandas dataframe into list for easy list parsing

In [4]:

queries_dataset = pd.read_csv("./datasets/customer_queries.csv")
queries_dataset = queries_dataset['Query'].to_list()


In [5]:
"""Function to structure the response as the responses may be unstructure
- A class SupportCase deriving from Pydantic BaseModel
- Parsers to parse the response and convert it into a well-defined structured Support Ticket item.
"""

import re
from pydantic import BaseModel, Field


class SupportCase(BaseModel):
    Query: str
    Title: str
    Browser: str
    OS: str
    Customer_Type: str
    Issue: str
    Resolution: str


def parse_unstructured_response(response: str) -> SupportCase:
    """Define the patterns for the parser and later parse the response to convert it to the desired format
    Rules for parsing: 
        1. If double quotes (") are before newline there then it stops until double quotes
        2. If comma (,) is before newline and no double quotes then it stops until comma
        3. If both are present (, and ") it stops until double quotes.
        4. If both are absent then the parser stops at the newline

    """
    patterns = {
        "Query": r'"Query":\s*"([^"\n,]*)(?:(?<=,)[^"\n]*|(?<=\n)[^"]*)?',
        "Title": r'"Title":\s*"([^"\n,]*)(?:(?<=,)[^"\n]*|(?<=\n)[^"]*)?',
        "Browser": r'"Browser":\s*"([^"\n,]*)(?:(?<=,)[^"\n]*|(?<=\n)[^"]*)?',
        "OS": r'"OS":\s*"([^"\n,]*)(?:(?<=,)[^"\n]*|(?<=\n)[^"]*)?',
        "Customer_Type": r'"Customer Type":\s*"([^"\n,]*)(?:(?<=,)[^"\n]*|(?<=\n)[^"]*)?',
        "Issue": r'"Issue":\s*"([^"\n,]*)(?:(?<=,)[^"\n]*|(?<=\n)[^"]*)?',
        "Resolution": r'"Resolution":\s*"([^"\n,]*)(?:(?<=,)[^"\n]*|(?<=\n)[^"]*)?'
    }
    parsed_data = {key: re.search(pattern, response, re.DOTALL).group(1).strip() if re.search(pattern, response, re.DOTALL) else "" for key, pattern in patterns.items()}
    return SupportCase(**parsed_data)


In [6]:

from tqdm import tqdm
def generate_synthetic_dataset(query): 
    """Generate the synthetic data
    - Parse the response to a structured SupportCase object
    - Convert the SupportCase object into a dictionary
    - return the dictionary
    """   

    response = generate_support_ticket(query)
    support_ticket = parse_unstructured_response(response)
    support_ticket = support_ticket.model_dump()
    return support_ticket



In [7]:
synthetic_data = []
for query in tqdm(queries_dataset, desc = "Generating Synthetic Data .. "):
    data = generate_synthetic_dataset(query)
    print(data)
    synthetic_data.append(data)  

Generating Synthetic Data .. :   0%|          | 0/40 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Synthetic Data .. :   2%|▎         | 1/40 [00:39<25:25, 39.12s/it]

{'Query': "I'm unable to generate reports from the dashboard. The button remains greyed out even after selecting all the required filters. This happens on Chrome on macOS Sonoma.", 'Title': 'Dashboard report generation button is greyed out after applying filters', 'Browser': 'Chrome', 'OS': 'macOS Sonoma', 'Customer_Type': 'Individual', 'Issue': 'Dashboard report generation button remains greyed out after applying filters', 'Resolution': 'Clear browser cache and cookies'}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Generating Synthetic Data .. :  98%|█████████▊| 39/40 [20:43<00:31, 31.67s/it]

{'Query': 'We tried sending a bulk invite to users but only some received the email. We’re using the admin console. Why?', 'Title': 'Bulk invite not reaching all users in admin console', 'Browser': '', 'OS': '', 'Customer_Type': 'Enterprise', 'Issue': 'Bulk invite not reaching all users in admin console', 'Resolution': 'Check email server settings and verify recipient list for bulk invites'}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Synthetic Data .. : 100%|██████████| 40/40 [21:07<00:00, 31.68s/it]

{'Query': 'I accidentally deleted an important workspace. Is there a way to restore it? It happened an hour ago. – Maria', 'Title': 'Workspace deletion and restoration request', 'Browser': '', 'OS': '', 'Customer_Type': 'Individual', 'Issue': 'Accidental deletion of workspace and need to restore it', 'Resolution': 'Check if a backup exists and restore it'}


In [8]:
synthetic_data

[{'Query': "I'm unable to generate reports from the dashboard. The button remains greyed out even after selecting all the required filters. This happens on Chrome on macOS Sonoma.",
  'Title': 'Dashboard report generation button is greyed out after applying filters',
  'Browser': 'Chrome',
  'OS': 'macOS Sonoma',
  'Customer_Type': 'Individual',
  'Issue': 'Dashboard report generation button remains greyed out after applying filters',
  'Resolution': 'Clear browser cache and cookies'},
 {'Query': "Our team can't access shared folders anymore. We keep getting a 'Permission Denied' error",
  'Title': 'Shared folder access denied error on Firefox on Windows 10',
  'Browser': 'Firefox',
  'OS': 'Windows 10',
  'Customer_Type': 'Enterprise',
  'Issue': 'Permission denied error when accessing shared folders despite no role changes',
  'Resolution': 'Check firewall settings and ensure the shared folder permissions are correctly configured'},
 {'Query': "The mobile app crashes on launch for An

In [9]:
# Check if synthetic data is of type json object
import json

def is_json_serializable(data):
    """Functioin to check if the synthetic data is of type json object. So that it can be converted easily to a csv file using pandas"""

    try:
        json.dumps(data)
        return True
    except (TypeError, OverflowError):
        return False
    
is_json_serializable(synthetic_data)

True

In [10]:
import pandas as pd

data = synthetic_data
csv_filename = "./datasets/query_support_ticket.csv"

df = pd.DataFrame(data)
df.to_csv(csv_filename, index=False)